In [1]:
from sklearn.svm import SVC
from pennylane import numpy as np
import pennylane as qml
from model_parameters import ModelParameters
from helper.scores import Metrics
from helper.svm import run_svm_sequence

# Kernel definition

In [ ]:
n_qubits = 8

dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    qml.AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(qml.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])


# Training QSVM

In [2]:
metrics = Metrics()

model_parameters = ModelParameters()

# Initialize the model
model = SVC(kernel=kernel_matrix, C=1.0) 

x_train, x_test, y_train, y_test = model_parameters.get_htru_2()

run_svm_sequence(model_parameters, model, x_train, y_train, x_test, y_test, metrics)

metrics.display()

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [58:43<00:00, 587.27s/it]

Pour 6 runs:
Accuracy: 0.965 ± 0.017
Balanced Accuracy: 0.970 ± 0.011
Recall: 0.977 ± 0.017
Specificity: 0.963 ± 0.019
Precision: 0.743 ± 0.105
NPV: 0.998 ± 0.002
G-Mean: 0.970 ± 0.011
Informedness: 0.940 ± 0.022
Training Duration: 215.773 ± 3.487
Testing Duration: 371.481 ± 5.779
